# Text generation with an RNN

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Below is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but here are some things to consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [29]:
import os
import time

import numpy as np
import tensorflow as tf

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [30]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

### Read the data

First, we'll download the file and then decode.

In [31]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


Let's take a look at the first 250 characters in text

In [ ]:
print(text[:250])

Let's check to see how many unique characters are in our corpus/document.

In [33]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

Using `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [26]:


# TODO 1
chars = tf.strings.unicode_split(vocab, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'\n'],
 [b' '],
 [b'!'],
 [b'$'],
 [b'&'],
 [b"'"],
 [b','],
 [b'-'],
 [b'.'],
 [b'3'],
 [b':'],
 [b';'],
 [b'?'],
 [b'A'],
 [b'B'],
 [b'C'],
 [b'D'],
 [b'E'],
 [b'F'],
 [b'G'],
 [b'H'],
 [b'I'],
 [b'J'],
 [b'K'],
 [b'L'],
 [b'M'],
 [b'N'],
 [b'O'],
 [b'P'],
 [b'Q'],
 [b'R'],
 [b'S'],
 [b'T'],
 [b'U'],
 [b'V'],
 [b'W'],
 [b'X'],
 [b'Y'],
 [b'Z'],
 [b'a'],
 [b'b'],
 [b'c'],
 [b'd'],
 [b'e'],
 [b'f'],
 [b'g'],
 [b'h'],
 [b'i'],
 [b'j'],
 [b'k'],
 [b'l'],
 [b'm'],
 [b'n'],
 [b'o'],
 [b'p'],
 [b'q'],
 [b'r'],
 [b's'],
 [b't'],
 [b'u'],
 [b'v'],
 [b'w'],
 [b'x'],
 [b'y'],
 [b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [34]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

It converts from tokens to character IDs:

In [35]:
ids = ids_from_chars(vocab)
ids

<tf.Tensor: shape=(65,), dtype=int64, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65])>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [22]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [36]:
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(65,), dtype=string, numpy=
array([b'\n', b' ', b'!', b'$', b'&', b"'", b',', b'-', b'.', b'3', b':',
       b';', b'?', b'A', b'B', b'C', b'D', b'E', b'F', b'G', b'H', b'I',
       b'J', b'K', b'L', b'M', b'N', b'O', b'P', b'Q', b'R', b'S', b'T',
       b'U', b'V', b'W', b'X', b'Y', b'Z', b'a', b'b', b'c', b'd', b'e',
       b'f', b'g', b'h', b'i', b'j', b'k', b'l', b'm', b'n', b'o', b'p',
       b'q', b'r', b's', b't', b'u', b'v', b'w', b'x', b'y', b'z'],
      dtype=object)>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [38]:
tf.strings.reduce_join(chars, axis=-1).numpy()

b"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [39]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

First use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [53]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [55]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

The class below does the following:
- We derive a class from tf.keras.Model
- The constructor is used to define the layers of the model
- We define the pass forward using the layers defined in the constructor

In [56]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [57]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [70]:
for input_example_batch, target_example_batch in dataset.take(3):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)
(64, 100, 66) # (batch_size, sequence_length, vocab_size)
(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [59]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [60]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [61]:
sampled_indices

array([33, 51, 32, 54,  4, 48, 47, 35, 49, 60, 41, 63, 49,  0,  6, 12, 20,
        2, 10, 14, 62, 46, 55, 13, 54, 38, 34, 11, 24, 21, 35, 18,  8, 50,
       18, 10, 17, 16,  6, 35, 36, 19, 42, 46, 10, 55,  4, 61,  8, 31,  2,
       49,  5,  4,  7, 35,  0, 18, 41,  1, 18, 56, 37, 64,  1, 39, 25, 29,
       57,  4, 51,  7,  0,  2, 32, 16, 33,  4, 23, 35, 11, 35, 12, 29,  7,
       37, 44, 34, 48,  8, 40, 35, 31, 14,  0, 64, 60,  6,  5,  7])

Decode these to see the text predicted by this untrained model:

In [62]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"s birth of love it is to me,\nThat I must love a loathed enemy.\n\nNurse:\nWhat's this? what's this?\n\nJU"

Next Char Predictions:
 b"TlSo$ihVjubxj[UNK]';G 3Awgp?oYU:KHVE-kE3DC'VWFcg3p$v-R j&$,V[UNK]Eb\nEqXy\nZLPr$l,[UNK] SCT$JV:V;P,XeUi-aVRA[UNK]yu'&,"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

#### Attach an optimizer, and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [63]:
# add a loss function here
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [64]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.191855, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [65]:
tf.exp(example_batch_mean_loss).numpy()

66.14537

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [66]:
model.compile(optimizer="adam", loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [67]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [68]:
EPOCHS = 10

In [69]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 16s 58ms/step - loss: 2.7370
Epoch 2/10
172/172 [==============================] - 11s 52ms/step - loss: 1.9959
Epoch 3/10
172/172 [==============================] - 11s 53ms/step - loss: 1.7185
Epoch 4/10
172/172 [==============================] - 12s 51ms/step - loss: 1.5578
Epoch 5/10
172/172 [==============================] - 11s 51ms/step - loss: 1.4589
Epoch 6/10
172/172 [==============================] - 11s 52ms/step - loss: 1.3904
Epoch 7/10
172/172 [==============================] - 12s 52ms/step - loss: 1.3383
Epoch 8/10
172/172 [==============================] - 12s 54ms/step - loss: 1.2919
Epoch 9/10
172/172 [==============================] - 13s 53ms/step - loss: 1.2520
Epoch 10/10
172/172 [==============================] - 12s 53ms/step - loss: 1.2126


#### Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [71]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [72]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [76]:
start = time.time()
states = None
next_char = tf.constant(["King:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

King:
He will fare far so: and these alls,
Is nonce to Jove, but being relate,
And not a suffers, but never be your
swit, was when their crew't all, on her infects rut of with?
'Tis first before your cozentray'a coming:
But tell me, toward the friend,
Which doibly think that bear the happiness:
Coast of the time to change it, duty.
He cannot bids the condract of his fly.
The children read what it may heart
The raspasting summer back and end.

HOSTENSIO:
Pray you, the honourable state as a subjects,
With sorrow his keen the pursuishment.
By the world, even their duty that why, follow;
If I did Marcius, there behoved from hence,
Nor which is the footate, come betwern,
Than your father rather like me see
this sharing the law, and let these precious
Tell-house they terms and kills of themberland!

Marince:
Amen. God, knowledge with valition!

LADY CAPULET:
Hair as the Forth gentle Edward; and theirs you cannot be a puspice?

DUCHESS OF YORK:
And every mercy you well accidens alieve
Bolingb

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [78]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [79]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Lave, then, By authority, for month
her charity.

ROMEO:
She is!

KING HENRY VI:
Send themselves, w


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [80]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {"loss": loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [81]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [82]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [83]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 52ms/step - loss: 2.7040


Or if you need more control, you can write your own complete custom training loop:

In [84]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for batch_n, (inp, target) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs["loss"])

        if batch_n % 50 == 0:
            template = (
                f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            )
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f"Epoch {epoch+1} Loss: {mean.result().numpy():.4f}")
    print(f"Time taken for 1 epoch {time.time() - start:.2f} sec")
    print("_" * 80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1954
Epoch 1 Batch 50 Loss 2.0525
Epoch 1 Batch 100 Loss 1.9128
Epoch 1 Batch 150 Loss 1.8790

Epoch 1 Loss: 1.9822
Time taken for 1 epoch 20.47 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7968
Epoch 2 Batch 50 Loss 1.7716
Epoch 2 Batch 100 Loss 1.6157
Epoch 2 Batch 150 Loss 1.6112

Epoch 2 Loss: 1.7019
Time taken for 1 epoch 10.36 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5920
Epoch 3 Batch 50 Loss 1.5564
Epoch 3 Batch 100 Loss 1.5910
Epoch 3 Batch 150 Loss 1.5049

Epoch 3 Loss: 1.5403
Time taken for 1 epoch 10.26 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4568
Epoch 4 Batch 50 Loss 1.4069
Epoch 4 Batch 100 Loss 1.4827
Epoch 4 Batch 150 Loss 1.4252

Epoch 4 Loss: 1.4430
Time taken for 1 epoch 10.45 sec
_____________________________________________________________________